In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.optimizers import Adam

In [2]:
train_path ='/Users/arghaauddy/Downloads/tomato/train'
val_path='/Users/arghaauddy/Downloads/tomato/val'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Apply ImageDataGenerator functionality to Trainset and Test set
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(256, 256),
    batch_size=80,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(256, 256),
    batch_size=80,
    class_mode='categorical'
)

Found 10000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [4]:
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Adding Dense Layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1000, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# The final model
model = Model(inputs=base_model.input, outputs=predictions)

# Configure the Learning Process
for layers in base_model.layers[:140]:
    layers.trainable=False
for layers in base_model.layers[140:]:
    layers.trainable=True

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=2
)

# Save the Model
model.save('tomato_disease_detector.h5')

Epoch 1/2


/Users/arghaauddy/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 4341s 35s/step - accuracy: 0.5093 - loss: 1.5584 - val_accuracy: 0.8470 - val_loss: 0.4740
Epoch 2/2
125/125 ━━━━━━━━━━━━━━━━━━━━ 5876s 47s/step - accuracy: 0.9186 - loss: 0.2640 - val_accuracy: 0.9320 - val_loss: 0.1846


In [6]:
# Import additional libraries for handling image input
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('tomato_disease_detector.h5')

# Define a function to preprocess the input image and make a prediction
def predict_disease(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    predicted_class = np.argmax(prediction, axis=1)[0]
    disease_name = class_indices[predicted_class]

    return disease_name

# Test the function
img_path = '/Users/arghaauddy/Downloads/tomato/val/Tomato___Tomato_Yellow_Leaf_Curl_Virus/1af87bdf-0bd0-4146-93e5-cc5f97d98b05___UF.GRC_YLCV_Lab 01968.JPG' # Replace with the path to your test image
disease_name = predict_disease(img_path)
print(f"The disease detected is: {disease_name}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
The disease detected is: Tomato___Tomato_Yellow_Leaf_Curl_Virus


In [7]:
pwd


'/Users/arghaauddy'